<a href="https://colab.research.google.com/github/jerrysdatanote01-collab/Learn-Pytorch/blob/main/Neural_Network_Pytorch_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np

In [2]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [3]:
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [5]:
class MyDataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x).float()
    self.y = torch.tensor(y).float()

  def __len__(self):
    return len(self.x)

  def __getitem__(self, ix):
    return self.x[ix], self.y[ix]

In [6]:
ds = MyDataset(X, Y)

/tmp/ipython-input-500267887.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
/tmp/ipython-input-500267887.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [7]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [8]:
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)).to(device)

In [9]:
!pip install torch_summary
from torchsummary import summary
summary(model)

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            24
├─ReLU: 1-2                              --
├─Linear: 1-3                            9
Total params: 33
Trainable params: 33
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            24
├─ReLU: 1-2                              --
├─Linear: 1-3                            9
Total params: 33
Trainable params: 33
Non-trainable params: 0

In [10]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)

In [11]:
import time
loss_history = []
start = time.time()
for _ in range(50):
  for ix,iy in dl:
    opt.zero_grad()
    loss_value = loss_func(model(ix),iy)
    loss_value.backward()
    opt.step()
    loss_history.append(loss_value.item())
end = time.time()
print(end - start)

0.1226963996887207


In [12]:
val = [[8,9],[10,11],[1.5,2.5]]
model(torch.tensor(val).float().to(device))

tensor([[16.9390],
        [20.8966],
        [ 4.0770]], grad_fn=<AddmmBackward0>)

In [13]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.1322,  0.8144],
                      [ 0.4434,  0.1626],
                      [ 0.3579, -0.3719],
                      [ 0.3887,  0.7976],
                      [ 0.2058,  0.6826],
                      [ 0.3130,  0.1295],
                      [-0.1560,  0.5336],
                      [ 0.6450, -0.0684]])),
             ('0.bias',
              tensor([ 0.3711, -0.4798,  0.3720, -0.6425,  0.2154, -0.4265,  0.4223, -0.1096])),
             ('2.weight',
              tensor([[ 0.5238,  0.0825, -0.1761,  0.6199,  0.5476,  0.0804,  0.2492,  0.3811]])),
             ('2.bias', tensor([-0.2205]))])

In [14]:
torch.save(model.state_dict(), 'model.pth')

In [15]:
model_load = torch.load('model.pth')
model.load_state_dict(model_load)

<All keys matched successfully>